In [ ]:
import pandas as pd
import numpy as np
import pickle
import re
import os.path
import sys
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import Input, Concatenate, InputLayer
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings 
warnings.filterwarnings('always')
# Helper code files

sys.path.append('../')
from Utils.utils import get_dataset, get_original_architecture
from Utils.utils import get_alternate_architecture, set_model_weights, adversarial_examples
from Utils.adv_gen import adversarial_examples


In [ ]:
# Global Variables
input_shape = (224, 224, 3)
num_classes = 1000
batch_size = 64


In [ ]:
# GET ALL MODELS

# # ORIGINAL ARCHITECTURE OF VGG
# model = get_original_architecture()

# # Set model weights and as trainable or not
# original_model = set_model_weights(model, "original")

# ALTERED ARCHITECTURE OF VGG FOR BLURRING
if os.path.isfile("Model Weights/blur_model_weights.h5"):
    blur_model = get_alternate_architecture()
    blur_model.load_weights("Model Weights/blur_model_weights.h5")
    print("Updating existing weights")
    
else:
    model = get_alternate_architecture()
    blur_model = set_model_weights(model, "blur")
    del model
    print("Setting up a new model")
# ALTERED ARCHITECTURE OF VGG
# model = get_alternate_architecture("alternate_architecture")
# alt_model = set_model_weights(model, "blur")



In [ ]:
# Get the train and test data generators
test_data = get_dataset(batch_size, True)


In [ ]:
def create_dataset(generator, train_type='blur'):
    
    if train_type=='blur':
        t1 = tf.convert_to_tensor(generator[0], dtype=tf.float32)
        t2 = tf.convert_to_tensor(generator[1], dtype=tf.float32)
        t3 = tf.convert_to_tensor(generator[2], dtype=tf.float32)
        t4 = tf.convert_to_tensor(generator[3], dtype=tf.float32)
        t5 = tf.convert_to_tensor(generator[4], dtype=tf.float32)
        
    else:
        t1 = t2 = t3 = t4 = t5 = tf.convert_to_tensor(generator[0], dtype=tf.float32)

    return (t1,t2,t3,t4,t5)
        

In [ ]:
train_data = get_dataset(batch_size, True)

# t1 = train_data[0].next()
        
# generator = (t1[0], train_data[1].next()[0], train_data[2].next()[0], train_data[3].next()[0],
#             train_data[4].next()[0])

# label = t1[1]


In [ ]:
# TO PLOT BLURRED IMAGE VERSIONS
# l = []
# l.append(train_data[0].next()[0][0])
# l.append(train_data[1].next()[0][0])
# l.append(train_data[2].next()[0][0])
# l.append(train_data[3].next()[0][0])
# l.append(train_data[4].next()[0][0])


# fig, ax = plt.subplots(1,5, figsize=(25,15))
# ax = ax.flatten()
# l = np.asarray(l)
# for i in range(5):
#     ax[i].imshow(l[i])
#     ax[i].set_xticks([])
#     ax[i].set_yticks([])
# # plt.savefig('../Documentation/Plots/blurred_2.png', format='png', dpi=1200, transparent=True)


In [ ]:
blur_model.compile(loss='categorical_crossentropy', metrics='accuracy', 
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

blur_model.evaluate(generator, label)


In [ ]:
model = get_original_architecture()

# Set model weights and as trainable or not
original_model = set_model_weights(model, "original")


In [ ]:
original_model.compile(loss='categorical_crossentropy', metrics='accuracy', 
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

original_model.evaluate(t1[0], label)